# Algorithm: Find The Lowest Value in an Array


In [2]:
number_list: list[int] = [1,2,3,4,5,6,7,8,9]
current_number: int = number_list[0]
for num in number_list:
    if num < current_number:
        current_number = num
print(current_number)    

1


In [1]:
 

# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from typing import Dict, Tuple

# Define the capacities of the buckets
BUCKET_CAPACITIES: Dict[str, int] = {
    '8': 8,
    '5': 5,
    '3': 3
}

# Initialize the current state of the buckets
buckets: Dict[str, int] = {
    '8': 0,
    '5': 0,
    '3': 0
}

# Goal volume
GOAL_VOLUME: int = 4

# Initialize the output area for displaying messages
output = widgets.Output()

# Function to update the visual representation of the buckets
def update_display():
    with output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(6, 8))
        ax.set_xlim(0, 4)
        ax.set_ylim(0, max(BUCKET_CAPACITIES.values()) + 2)
        ax.axis('off')
        bucket_positions = {'8': 1, '5': 2, '3': 3}
        colors = {'8': '#1f77b4', '5': '#ff7f0e', '3': '#2ca02c'}

        for label, capacity in BUCKET_CAPACITIES.items():
            current = buckets[label]
            x = bucket_positions[label]
            # Draw bucket outline
            ax.add_patch(Rectangle((x, 0), 0.8, capacity, fill=False, edgecolor='black', linewidth=2))
            # Draw water level
            ax.add_patch(Rectangle((x, 0), 0.8, current, color=colors[label]))
            # Add text labels
            ax.text(x + 0.4, capacity + 0.2, f'{capacity}L', ha='center', va='bottom', fontsize=12, fontweight='bold')
            ax.text(x + 0.4, current + 0.2, f'{current}L', ha='center', va='bottom', fontsize=12, color='white' if current > 0 else 'black')
            ax.text(x + 0.4, -1, f'{label}L Bucket', ha='center', va='top', fontsize=12, fontweight='bold')

        plt.title('Water Bucket Puzzle', fontsize=16, fontweight='bold')
        plt.show()

# Function to display messages
def display_message(message: str):
    with output:
        print(message)

# Action Functions
def fill_bucket_action(bucket: str):
    if buckets[bucket] == BUCKET_CAPACITIES[bucket]:
        display_message(f"Bucket {bucket}L is already full.")
    else:
        buckets[bucket] = BUCKET_CAPACITIES[bucket]
        display_message(f"Filled {bucket}L bucket.")
    update_display()
    check_goal()

def empty_bucket_action(bucket: str):
    if buckets[bucket] == 0:
        display_message(f"Bucket {bucket}L is already empty.")
    else:
        buckets[bucket] = 0
        display_message(f"Emptied {bucket}L bucket.")
    update_display()
    check_goal()

def pour_bucket_action(from_bucket: str, to_bucket: str):
    if buckets[from_bucket] == 0:
        display_message(f"Bucket {from_bucket}L is empty. Cannot pour.")
        return
    available_space: int = BUCKET_CAPACITIES[to_bucket] - buckets[to_bucket]
    if available_space == 0:
        display_message(f"Bucket {to_bucket}L is already full.")
        return
    pour_amount: int = min(buckets[from_bucket], available_space)
    buckets[from_bucket] -= pour_amount
    buckets[to_bucket] += pour_amount
    display_message(f"Poured {pour_amount}L from {from_bucket}L bucket to {to_bucket}L bucket.")
    update_display()
    check_goal()

def check_goal():
    if GOAL_VOLUME in buckets.values():
        display_message(f"🎉 Congratulations! You've achieved {GOAL_VOLUME} liters in one of the buckets. 🎉")
        # Disable all buttons after winning
        fill_buttons_box.disabled = True
        empty_buttons_box.disabled = True
        pour_buttons_box.disabled = True

# Create UI Components

# Fill Buttons
fill_buttons = []
for label in BUCKET_CAPACITIES.keys():
    btn = widgets.Button(description=f"Fill {label}L", button_style='success')
    btn.on_click(lambda b, lbl=label: fill_bucket_action(lbl))
    fill_buttons.append(btn)
fill_buttons_box = widgets.HBox(fill_buttons)

# Empty Buttons
empty_buttons = []
for label in BUCKET_CAPACITIES.keys():
    btn = widgets.Button(description=f"Empty {label}L", button_style='warning')
    btn.on_click(lambda b, lbl=label: empty_bucket_action(lbl))
    empty_buttons.append(btn)
empty_buttons_box = widgets.HBox(empty_buttons)

# Pour Buttons
pour_buttons = []
bucket_labels = list(BUCKET_CAPACITIES.keys())
for from_b in bucket_labels:
    for to_b in bucket_labels:
        if from_b != to_b:
            btn = widgets.Button(description=f"Pour {from_b}L → {to_b}L", button_style='info')
            btn.on_click(lambda b, frm=from_b, to=to_b: pour_bucket_action(frm, to))
            pour_buttons.append(btn)
pour_buttons_box = widgets.HBox(pour_buttons)

# Reset Button
def reset_game(b):
    for key in buckets.keys():
        buckets[key] = 0
    display_message("🔄 Game has been reset.")
    update_display()
    # Enable all buttons
    fill_buttons_box.disabled = False
    empty_buttons_box.disabled = False
    pour_buttons_box.disabled = False

reset_btn = widgets.Button(description="Reset Game", button_style='danger')
reset_btn.on_click(reset_game)

# Layout
action_title = widgets.HTML(value="<h3>Actions</h3>")
controls = widgets.VBox([
    action_title,
    widgets.Label("Fill a Bucket:"),
    fill_buttons_box,
    widgets.Label("Empty a Bucket:"),
    empty_buttons_box,
    widgets.Label("Pour Water Between Buckets:"),
    pour_buttons_box,
    reset_btn
])

# Display initial state
update_display()

# Final Layout
game_layout = widgets.HBox([controls, output])

display(game_layout)
